### Imports

In [ ]:
import cv2
from typing import Tuple,List
from math import sin,cos,pi
from skimage.draw import line as line_points
import numpy as np
from numpy.linalg import norm

### Constants

In [ ]:
WHITE = (255,255,255)
RED = (0,0,255)
GREEN = (0,255,0)
BLACK = (0,0,0)
ROBOT_BORDER=(6,6,6)
FILL = -1
ROBOT_RADIUS = 10
ROBOT_BORDER_THICKNESS = 2
DIRECTION_LINE_LENGTH = 20
RADIAN = pi/180.0
SENSOR_ANGLE = 250
SENSOR_RESOLUTION = 2

### Functions

In [ ]:
def xy_from_r_theta_xy(r:float,theta: float, x: int, y:int) -> Tuple[int,int]:
    '''
        Takes the starting point of a line, its slope angle and a length, and returns the ending point
    '''
    x2 = int(x + r * cos(theta *RADIAN))
    y2 = int(y + r * sin(theta *RADIAN))

    return x2,y2

def line(img: cv2.Mat,
         from_point: Tuple[int,int],
         with_length: float,
         at_angle : float,
         color: Tuple[int,int,int],
         thickness: int
         ) -> cv2.Mat:
    
    x,y = from_point
    x2,y2 = xy_from_r_theta_xy(with_length,at_angle,x,y)
    return cv2.line(img,from_point,(x2,y2),color,thickness)

def cast_ray(img: cv2.Mat,
             from_point: Tuple[int,int],
             with_max_length: int,
             at_angle: float,
             color: Tuple[int,int,int]) -> Tuple[int,int]:

    x1,y1 = from_point

    farthest_point = xy_from_r_theta_xy(with_max_length,at_angle,x1,y1)
    x2,y2 = farthest_point

    lpoints = list(zip(*line_points(x1,y1,x2,y2)))

    endpoint = None
    done = False
    for x,y in lpoints:
        # check if the difference between current pixel and black is less than 10
        # if norm(img[y,x] - np.array(BLACK)) < 10:
        if norm(img[y,x] - np.array(BLACK)) == 0 :
            done = True
            endpoint = (x,y)
            break
        if not done: endpoint = (x,y)
       
    
    img = cv2.line(img,(x1,y1),endpoint,color,thickness=1)
    return endpoint

def draw_robot(map, x, y, t):
    img = cv2.circle(map,
                    center= (x,y),
                    radius= ROBOT_RADIUS,
                    color = RED,
                    thickness=FILL)
    img = cv2.circle(img,
                    center= (x,y),
                    radius= ROBOT_RADIUS,
                    color = ROBOT_BORDER,
                    thickness=ROBOT_BORDER_THICKNESS)
    img = line(img,
                from_point= (x,y),
                with_length= DIRECTION_LINE_LENGTH,
                at_angle= t,
                color= ROBOT_BORDER,
                thickness= ROBOT_BORDER_THICKNESS)
    return img

def move_ray(distance: int, x: float, y: float, angle: int):
    return round(x + distance * np.cos(angle * np.pi / 180)), round(y + distance * np.sin(angle * np.pi / 180))

### Requirement 1

In [ ]:
def requirement1(image_path: str , x: int, y: int, t: int):
    '''
        This function is for requirement 1

        Parameters:
            image_path: str
                Path to the image
            x,y: int
                Location of Robot, (0,0) is the left-top corner of the 2D map, so x increases to the right and y increases to the down.
            t: int
                Angle of Robot's pose, 0 means robot is facing to the right, 90 means to the down, 180 to the left, 270 to the up.
    '''
    # Reading an image in color mode
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # write the position of the robot in txt file
    file = open("measurements.txt", "w")
    file.write("Robot position: " + "x: " + str(x) + " y: " + str(y) + " theta: " + str(t))

    # draw the robot
    image = draw_robot(image, x, y, t)

    # create array to save the distance at each angle
    distances = np.zeros(int(SENSOR_ANGLE/SENSOR_RESOLUTION)+1)
    endpoints = []

    index = 0

    # write meaeurements in txt file
    file.write("\n\nMeasurements: ") 
    file.write("\nAngle\tDistance\tEndpoint")


    for angle in range(t- int(SENSOR_ANGLE/ 2),t+ int(SENSOR_ANGLE/2) + 1,2):
        endpoint = cast_ray(image,
                            from_point= (x,y),
                            with_max_length= int(12 *100 / 4),
                            at_angle= angle,
                            color= GREEN)
                            

        # calculate the distance between the robot and the endpoint
        distance = norm(np.array((x,y)) - np.array(endpoint))

        # save the distance in the array
        distances[index] = min(1200, distance * 4)
        endpoints.append(endpoint)

        # write meaeurements in txt file
        file.write("\n" + str(angle) + "\t" + str(distances[index]) + "\t" + str(endpoint))
        
        # increase the index
        index += 1
    
    file.close()

    cv2.imwrite('req1.jpg', image)

    return distances , np.array(endpoints) 

### Requirement 2

In [ ]:
def requirement2(image_path: str , actual_measures: List[float] , endpoints: np.ndarray):
    '''
        This function is for requirement 2

        Parameters:
            image_path: str
                Path to the image
            actual_measures: List[float]
                Actual measures of the robot
    '''
    
    # read the map image and convert it to gray scale
    map = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # do binary thresholding
    binary_image = cv2.threshold(map, 127, 255, cv2.THRESH_BINARY)[1]

    # Apply the distance transform 
    likelihood_field = cv2.distanceTransform(binary_image, cv2.DIST_L2, 0)

    # let the error (sigma) = 20 cm ==> 5 pixels
    sigma = 5

    # pass the likelihood field through the gaussian function 
    likelihood_field = np.array(likelihood_field)
    likelihood_field = 1/np.sqrt(2*pi*sigma) * np.exp(-0.5*((likelihood_field)/sigma)**2)

    # max likelihood
    max_likelihood = np.max(likelihood_field)
   
    # Normalize all values to be between 0 and 1
    likelihood_field = likelihood_field/max_likelihood

    # output the likelihood field
    cv2.imwrite("likelihood_field.jpg",likelihood_field*255)

    # create an array to save the probabilities
    propbability_map = np.zeros(shape=(len(map),len(map[0])))

    # add the random value with a uniform distribution with probability 0.05
    likelihood_field = 0.9 * likelihood_field + 0.05 / (300 * max_likelihood )

    # calculate added value of max range of sensor
    max_range_added_value = 0.05 * 1/ (10 * max_likelihood)
 
    for y in range (0 , likelihood_field.shape[0] , 1):
        for x in range (0 , likelihood_field.shape[1] ,1 ):
            for theta in range (0,360 , 10):
                index = 0
                prob = 1
                for ray_angle in range (theta - int(SENSOR_ANGLE/2), theta + int(SENSOR_ANGLE/2) +1 , SENSOR_RESOLUTION):
                    x2,y2 = move_ray(actual_measures[index]/4,x,y,ray_angle)
                    
                    # if the endpoint is out of the map, then give it a very low probability ( 0.001 )
                    if x2 < 0 or x2 >= likelihood_field.shape[1] or y2 < 0 or y2 >= likelihood_field.shape[0]:        
                        prob *= 0.001
                    # if the endpoint in the max range of the sensor
                    elif actual_measures[index] >= 1160 :
                        prob *= likelihood_field[y2][x2] + max_range_added_value
                    else:
                        prob *= likelihood_field[y2][x2]                       
                    index += 1
                   
                # update the probability map if the probability is higher
                # than the current value in the map to guarantee that the
                # highest probability in all orientations given the position is saved
                propbability_map[y][x] = max(propbability_map[y][x],prob)
                
    # output the probability map
    cv2.imwrite("req2-probs.jpg",propbability_map* (255/ np.max(propbability_map)))

    # get the point with the highest probability with reverse order
    
    max_index = np.unravel_index(np.argmax(propbability_map, axis=None), propbability_map.shape)

    # print the value of the point with the highest probability
    print("The highest probability is: " + str(propbability_map[max_index[0]][max_index[1]]))

    # draw the point with the highest probability in the map
    max_point_map = np.zeros(shape=(len(map),len(map[0])))
    max_point_map[max_index] = 1
    cv2.imwrite("req2-highest.jpg",max_point_map* 255)
    
    print("The highest probability is at: ", (max_index[1],max_index[0]))  

In [ ]:
image_path = "Map.jpg"
distances , endponts = requirement1(image_path,350,185,200)
requirement2(image_path,distances , endponts)